In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()
from pyspark.ml.regression import LinearRegression
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/sejal@ibm.com/Ecommerce_Customers.csv", inferSchema=True, header=True)

In [0]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [0]:
# inspect a row of data
for col, item in zip(data.columns, data.head(1)[0]):
  print(col, ": ", item)


Email : mstephenson@fernandez.com
Address : 835 Frank TunnelWrightmouth, MI 82180-9605
Avatar : Violet
Avg Session Length : 34.49726772511229
Time on App : 12.65565114916675
Time on Website : 39.57766801952616
Length of Membership : 4.0826206329529615
Yearly Amount Spent : 587.9510539684005

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[10]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App',
 'Time on Website','Length of Membership'],outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
output.select('features').show()

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
[33.3306725236463...|
[33.8710378793419...|
[32.0215955013870...|
[32.7391429383803...|
[33.9877728956856...|
[31.9365486184489...|
[33.9925727749537...|
[33.8793608248049...|
[29.5324289670579...|
[33.1903340437226...|
[32.3879758531538...|
[30.7377203726281...|
[32.1253868972878...|
[32.3388993230671...|
[32.1878120459321...|
[32.6178560628234...|
+--------------------+
only showing top 20 rows

In [0]:
# what does our features vector look like?
output.head(1)[0][-1]

Out[15]: DenseVector([34.4973, 12.6557, 39.5777, 4.0826])

In [0]:
final_data = output.select('features', 'Yearly Amount Spent')

In [0]:
# train test split
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 346|
 mean| 502.807980168382|
 stddev| 78.36712847171002|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 154|
 mean| 491.4640129288004|
 stddev| 81.11265992147817|
 min| 266.086340948469|
 max| 744.2218671047146|
+-------+-------------------+

In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 10.982648235936495|
 -5.978037545084192|
 -17.61704207059222|
 -6.388255001902792|
 -21.93089116982094|
 9.520814510515493|
 -4.688634665882091|
 0.7950225341910482|
 -7.282166580308626|
 3.861085657098613|
 17.813173502789823|
 16.34670844946396|
-1.3795621216025893|
 -6.307672048384461|
 1.9847327968691957|
 -4.38761424519663|
 -11.41955646520114|
-18.151710351092845|
 7.348936509735438|
 -2.050515827181357|
+-------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[29]: 10.710489202276186

In [0]:
test_results.r2

Out[30]: 0.9824502627543185

In [0]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.4925366965402...|
[30.8162006488763...|
[31.0613251567161...|
[31.1239743499119...|
[31.1695067987115...|
[31.2681042107507...|
[31.3895854806643...|
[31.4474464941278...|
[31.5316044825729...|
[31.6005122003032...|
[31.6098395733896...|
[31.6610498227460...|
[31.7207699002873...|
[31.7366356860502...|
[31.7656188210424...|
[31.8093003166791...|
[31.8164283341993...|
[31.8209982016720...|
[31.8627411090001...|
+--------------------+
only showing top 20 rows

In [0]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 397.657702836691|
[30.4925366965402...| 288.4492832649987|
[30.8162006488763...| 283.7033830190612|
[31.0613251567161...| 493.9437130598044|
[31.1239743499119...| 508.8779450095867|
[31.1695067987115...| 417.8357162917773|
[31.2681042107507...| 428.159167839706|
[31.3895854806643...|409.27458852579184|
[31.4474464941278...|425.88490867553264|
[31.5316044825729...|432.65452007226395|
[31.6005122003032...| 461.3596779883071|
[31.6098395733896...| 428.1988412016442|
[31.6610498227460...| 417.7379157015034|
[31.7207699002873...| 545.0826055264074|
[31.7366356860502...| 494.9487134586627|
[31.7656188210424...|500.94169588080376|
[31.8093003166791...| 548.1914558280423|
[31.8164283341993...| 519.2742018547492|
[31.8209982016720...| 417.3263445034779|
[31.8627411090001...| 558.348657001228|
+--------------------+------------------+
only showing top 20 rows